In [ ]:
%matplotlib inline


Prepare Data
==============

In [ ]:
import os
import pandas as pd
os.getcwd()
os.chdir("/Users/chenyuening/Desktop/Overtone Health/AirForce/")

In [ ]:
data_file = 'news_clean.csv'
data = pd.read_csv(data_file)

In [ ]:
data.head()

,clean_content
0,WASHINGTON Congressional Republicans have ...
1,After the bullet shells get counted the blood ...
2,When Walt Disneys Bambi opened in 1942 critics...
3,Death may be the great equalizer but it isnt n...
4,SEOUL South Korea North Koreas leader Kim ...


In [ ]:
from gensim.utils import tokenize
from gensim import utils

class MyIter(object):
       def __iter__(self):
            path = datapath('/Users/chenyuening/Desktop/Overtone Health/AirForce/news_clean.csv')
            with utils.open(path, 'r', encoding='utf-8') as fin:
                for line in fin:
                    yield list(tokenize(line))


FastText Model
==============

Introduces Gensim's fastText model and demonstrates its use on the Lee Corpus.



In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Here, we'll learn to work with fastText library for training word-embedding
models, saving & loading them and performing similarity operations & vector
lookups analogous to Word2Vec.



When to use fastText?
---------------------

The main principle behind `fastText <https://github.com/facebookresearch/fastText>`_ is that the
morphological structure of a word carries important information about the meaning of the word.
Such structure is not taken into account by traditional word embeddings like Word2Vec, which
train a unique word embedding for every individual word.
This is especially significant for morphologically rich languages (German, Turkish) in which a
single word can have a large number of morphological forms, each of which might occur rarely,
thus making it hard to train good word embeddings.


fastText attempts to solve this by treating each word as the aggregation of its subwords.
For the sake of simplicity and language-independence, subwords are taken to be the character ngrams
of the word. The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams.


According to a detailed comparison of Word2Vec and fastText in
`this notebook <https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb>`__,
fastText does significantly better on syntactic tasks as compared to the original Word2Vec,
especially when the size of the training corpus is small. Word2Vec slightly outperforms fastText
on semantic tasks though. The differences grow smaller as the size of the training corpus increases.


fastText can obtain vectors even for out-of-vocabulary (OOV) words, by summing up vectors for its
component char-ngrams, provided at least one of the char-ngrams was present in the training data.




Training models
---------------




For the following examples, we'll use the Lee Corpus (which you already have if you've installed Gensim) for training our model.






In [ ]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [ ]:
model3 = FastText(size=100, window=3, min_count=1)
model3.build_vocab(sentences=MyIter())
total_examples = model3.corpus_count
model3.train(sentences=MyIter(), total_examples=total_examples, epochs=5)

2021-03-03 13:43:38,891 : INFO : resetting layer weights
2021-03-03 13:43:46,367 : INFO : collecting all words and their counts
2021-03-03 13:43:46,386 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-03-03 13:43:46,578 : INFO : collected 14427 word types from a corpus of 111539 raw words and 102 sentences
2021-03-03 13:43:46,580 : INFO : Loading a fresh vocabulary
2021-03-03 13:43:46,698 : INFO : effective_min_count=1 retains 14427 unique words (100% of original 14427, drops 0)
2021-03-03 13:43:46,700 : INFO : effective_min_count=1 leaves 111539 word corpus (100% of original 111539, drops 0)
2021-03-03 13:43:46,754 : INFO : deleting the raw counts dictionary of 14427 items
2021-03-03 13:43:46,755 : INFO : sample=0.001 downsamples 37 most-common words
2021-03-03 13:43:46,756 : INFO : downsampling leaves estimated 88381 word corpus (79.2% of prior 111539)
2021-03-03 13:43:46,938 : INFO : estimated required memory for 14427 words, 95441 buckets and 100 dime

In [ ]:
wv = model3.wv
print(wv)

#
# FastText models support vector lookups for out-of-vocabulary words by summing up character ngrams belonging to the word.
#
print(wv.vocab)

{'A': <gensim.models.keyedvectors.Vocab object at 0x1a1c38dfd0>,
 'ABC': <gensim.models.keyedvectors.Vocab object at 0x1a24ac8f10>,
 'ABCs': <gensim.models.keyedvectors.Vocab object at 0x1a24aa0c90>,
 'AIDS': <gensim.models.keyedvectors.Vocab object at 0x1a1bab9710>,
 'AND': <gensim.models.keyedvectors.Vocab object at 0x1a1be9fc50>,
 'ANGELES': <gensim.models.keyedvectors.Vocab object at 0x1a1c45fd50>,
 'Aaron': <gensim.models.keyedvectors.Vocab object at 0x1a24a978d0>,
 'Abadi': <gensim.models.keyedvectors.Vocab object at 0x1a24a93510>,
 'Abbas': <gensim.models.keyedvectors.Vocab object at 0x1a1cc20990>,
 'Abdulkarim': <gensim.models.keyedvectors.Vocab object at 0x1a1bacf150>,
 'Abe': <gensim.models.keyedvectors.Vocab object at 0x1a1cc202d0>,
 'Abed': <gensim.models.keyedvectors.Vocab object at 0x1a1c47fb90>,
 'Abella': <gensim.models.keyedvectors.Vocab object at 0x1a1be1d8d0>,
 'Abenakis': <gensim.models.keyedvectors.Vocab object at 0x1a1c55a6d0>,
 'Abnaquis': <gensim.models.keyedvec

Training hyperparameters
^^^^^^^^^^^^^^^^^^^^^^^^




Hyperparameters for training the model follow the same pattern as Word2Vec. FastText supports the following parameters from the original word2vec:

- model: Training architecture. Allowed values: `cbow`, `skipgram` (Default `cbow`)
- vector_size: Dimensionality of vector embeddings to be learnt (Default 100)
- alpha: Initial learning rate (Default 0.025)
- window: Context window size (Default 5)
- min_count: Ignore words with number of occurrences below this (Default 5)
- loss: Training objective. Allowed values: `ns`, `hs`, `softmax` (Default `ns`)
- sample: Threshold for downsampling higher-frequency words (Default 0.001)
- negative: Number of negative words to sample, for `ns` (Default 5)
- epochs: Number of epochs (Default 5)
- sorted_vocab: Sort vocab by descending frequency (Default 1)
- threads: Number of threads to use (Default 12)


In addition, fastText has three additional parameters:

- min_n: min length of char ngrams (Default 3)
- max_n: max length of char ngrams (Default 6)
- bucket: number of buckets used for hashing ngrams (Default 2000000)


Parameters ``min_n`` and ``max_n`` control the lengths of character ngrams that each word is broken down into while training and looking up embeddings. If ``max_n`` is set to 0, or to be lesser than ``min_n``\ , no character ngrams are used, and the model effectively reduces to Word2Vec.



To bound the memory requirements of the model being trained, a hashing function is used that maps ngrams to integers in 1 to K. For hashing these character sequences, the `Fowler-Noll-Vo hashing function <http://www.isthe.com/chongo/tech/comp/fnv>`_ (FNV-1a variant) is employed.




**Note:** You can continue to train your model while using Gensim's native implementation of fastText.




Saving/loading models
---------------------




Models can be saved and loaded via the ``load`` and ``save`` methods, just like
any other model in Gensim.




In [ ]:
# Save a model trained via Gensim's fastText implementation to temp.
import tempfile
import os
with tempfile.NamedTemporaryFile(prefix='saved_model_gensim-', delete=False) as tmp:
    model.save(tmp.name, separately=[])

# Load back the same model.
loaded_model = FastText.load(tmp.name)
print(loaded_model)

os.unlink(tmp.name)  # demonstration complete, don't need the temp file anymore

2021-03-03 13:43:57,643 : INFO : saving FastText object under /var/folders/n5/_b0vf4854x9f2b7qzq6jfl6c0000gn/T/saved_model_gensim-arwpgckf, separately []
2021-03-03 13:43:57,645 : INFO : storing np array 'vectors_ngrams' to /var/folders/n5/_b0vf4854x9f2b7qzq6jfl6c0000gn/T/saved_model_gensim-arwpgckf.wv.vectors_ngrams.npy
2021-03-03 13:44:05,427 : INFO : not storing attribute vectors_norm
2021-03-03 13:44:05,428 : INFO : not storing attribute vectors_vocab_norm
2021-03-03 13:44:05,429 : INFO : not storing attribute vectors_ngrams_norm
2021-03-03 13:44:05,430 : INFO : not storing attribute buckets_word
2021-03-03 13:44:05,431 : INFO : storing np array 'vectors_ngrams_lockf' to /var/folders/n5/_b0vf4854x9f2b7qzq6jfl6c0000gn/T/saved_model_gensim-arwpgckf.trainables.vectors_ngrams_lockf.npy
2021-03-03 13:44:07,495 : INFO : saved /var/folders/n5/_b0vf4854x9f2b7qzq6jfl6c0000gn/T/saved_model_gensim-arwpgckf
2021-03-03 13:44:07,497 : INFO : loading FastText object from /var/folders/n5/_b0vf4854

The ``save_word2vec_format`` is also available for fastText models, but will
cause all vectors for ngrams to be lost.
As a result, a model loaded in this way will behave as a regular word2vec model.




Word vector lookup
------------------


All information necessary for looking up fastText words (incl. OOV words) is
contained in its ``model.wv`` attribute.

If you don't need to continue training your model, you can export & save this `.wv`
attribute and discard `model`, to save space and RAM.




In [ ]:
print('Trump' in wv.vocab)

True


In [ ]:
print(wv['Trump'])

array([ 4.30193484e-01, -6.80136740e-01, -4.24004823e-01, -2.14510784e-01,
       -2.51856502e-02, -1.79971188e-01, -4.03071046e-02, -1.38440654e-01,
        2.01387838e-01, -7.35689104e-02,  2.58425653e-01, -2.61822790e-01,
        2.23683402e-01, -8.22224596e-04, -9.69733670e-02,  6.65448248e-01,
        1.17647015e-01,  8.97217155e-01, -3.91626030e-01,  3.27698708e-01,
       -6.06435120e-01,  3.75649303e-01,  3.20760995e-01,  6.32308900e-01,
       -3.46562415e-01, -4.48793679e-01,  5.20175397e-01, -1.07146725e-01,
       -1.19446546e-01,  1.43746763e-01,  6.38738513e-01,  8.75252858e-03,
       -1.31779373e+00, -2.84384161e-01,  5.79379618e-01, -2.42836908e-01,
        5.59470832e-01,  3.92757118e-01, -9.58136976e-01,  2.77912229e-01,
       -3.29154819e-01,  1.32302806e-01, -8.70481655e-02, -8.11342970e-02,
       -8.80920053e-01, -8.73256266e-01, -2.49074459e-01, -7.37710178e-01,
        5.40574789e-01, -7.09867656e-01, -5.48335731e-01, -1.30136237e-01,
        2.63183445e-01,  

In [ ]:
print(wv['crime'])

array([ 0.34564787, -0.53944564, -0.34745598, -0.16806808, -0.03539889,
       -0.13148475, -0.02385111, -0.12018251,  0.16223428, -0.07700369,
        0.23557599, -0.21548198,  0.1940412 , -0.04205749, -0.09060662,
        0.53759   ,  0.08024418,  0.7417488 , -0.30788583,  0.27088577,
       -0.49637094,  0.31917363,  0.25429112,  0.5241335 , -0.2920705 ,
       -0.3702198 ,  0.42063552, -0.09139873, -0.11316667,  0.13735801,
        0.5197463 , -0.01922516, -1.0690329 , -0.21447036,  0.4573538 ,
       -0.21164367,  0.45763308,  0.30757293, -0.7563776 ,  0.19686303,
       -0.26283354,  0.12821619, -0.0819421 , -0.0515007 , -0.71365845,
       -0.71485513, -0.21563464, -0.5933293 ,  0.44172642, -0.5898562 ,
       -0.45314732, -0.11140038,  0.22846082,  0.5689621 ,  0.33275664,
        0.25854495, -0.02782949, -0.21491757, -0.54035974,  0.08178576,
        0.4175558 ,  0.6197923 ,  0.77983385,  0.01165168, -0.01568669,
       -0.27163073, -0.2952024 , -0.18414995, -1.0534136 ,  0.33

Similarity operations
---------------------




Similarity operations work the same way as word2vec. **Out-of-vocabulary words can also be used, provided they have at least one character ngram present in the training data.**




In [ ]:
print("Terrorist" in wv.vocab)

True


In [ ]:
print("Terrorism" in wv.vocab)

False


In [ ]:
print(wv.similarity("Terrorism", "Terrorist"))

0.99998665


In [ ]:
print("Air Force" in wv.vocab)

False


Syntactically similar words generally have high similarity in fastText models, since a large number of the component char-ngrams will be the same. As a result, fastText generally does better at syntactic tasks than Word2Vec. A detailed comparison is provided `here <Word2Vec_FastText_Comparison.ipynb>`_.




Other similarity operations
^^^^^^^^^^^^^^^^^^^^^^^^^^^

The example training corpus is a toy corpus, results are not expected to be good, for proof-of-concept only



In [ ]:
print(wv.most_similar("Trump"))

2021-03-03 13:46:38,707 : INFO : precomputing L2-norms of word weight vectors
2021-03-03 13:46:38,751 : INFO : precomputing L2-norms of ngram weight vectors


[('Trumps', 0.9999774694442749),
 ('Trumka', 0.9999765157699585),
 ('lump', 0.9999308586120605),
 ('Dump', 0.9999115467071533),
 ('slump', 0.9998789429664612),
 ('jump', 0.9998743534088135),
 ('grumpy', 0.9998619556427002),
 ('frumpy', 0.9998536109924316),
 ('Truth', 0.9997895359992981),
 ('Obama', 0.9997788667678833)]


In [ ]:
print(wv.most_similar("crime"))

[('prime', 0.9999914765357971),
 ('crimes', 0.9999900460243225),
 ('criminal', 0.9999892711639404),
 ('experiment', 0.9999890923500061),
 ('primer', 0.9999890327453613),
 ('daughters', 0.9999887943267822),
 ('stickers', 0.9999887347221375),
 ('cornerstone', 0.9999886751174927),
 ('socialistic', 0.9999884366989136),
 ('discriminatory', 0.999988317489624)]


In [ ]:
#print(wv.n_similarity(['sushi', 'shop'], ['japanese', 'restaurant']))

In [ ]:
#print(wv.doesnt_match("breakfast cereal dinner lunch".split()))

In [ ]:
print(wv.most_similar(positive=['crime', 'Terroist'], negative=['peace']))

[('Terrorist', 0.9999771118164062),
 ('Lawyers', 0.9999743700027466),
 ('Anderson', 0.999974250793457),
 ('businessman', 0.9999741911888123),
 ('prime', 0.9999738335609436),
 ('pronouns', 0.9999736547470093),
 ('momentous', 0.9999735951423645),
 ('overtime', 0.999973475933075),
 ('Celine', 0.9999732971191406),
 ('primer', 0.9999731779098511)]


In [ ]:
print(wv.evaluate_word_analogies(datapath('questions-words.txt')))

2021-03-03 13:48:32,628 : INFO : Evaluating word analogies for top 300000 words in the model on /opt/anaconda3/lib/python3.7/site-packages/gensim/test/test_data/questions-words.txt
2021-03-03 13:48:32,699 : INFO : capital-common-countries: 0.0% (0/72)
2021-03-03 13:48:32,799 : INFO : capital-world: 0.0% (0/168)
2021-03-03 13:48:32,809 : INFO : currency: 0.0% (0/12)
2021-03-03 13:48:32,915 : INFO : city-in-state: 0.0% (0/250)
2021-03-03 13:48:32,989 : INFO : family: 9.5% (20/210)
2021-03-03 13:48:33,084 : INFO : gram1-adjective-to-adverb: 53.7% (146/272)
2021-03-03 13:48:33,122 : INFO : gram2-opposite: 70.0% (63/90)
2021-03-03 13:48:33,341 : INFO : gram3-comparative: 19.2% (135/702)
2021-03-03 13:48:33,420 : INFO : gram4-superlative: 10.8% (26/240)
2021-03-03 13:48:33,571 : INFO : gram5-present-participle: 24.9% (126/506)
2021-03-03 13:48:33,697 : INFO : gram6-nationality-adjective: 19.3% (51/264)
2021-03-03 13:48:34,096 : INFO : gram7-past-tense: 2.2% (23/1056)
2021-03-03 13:48:34,201 

(0.16748329621380847,
 [{'correct': [],
   'incorrect': [('BAGHDAD', 'IRAQ', 'BEIJING', 'CHINA'),
                 ('BAGHDAD', 'IRAQ', 'BERLIN', 'GERMANY'),
                 ('BAGHDAD', 'IRAQ', 'KABUL', 'AFGHANISTAN'),
                 ('BAGHDAD', 'IRAQ', 'LONDON', 'ENGLAND'),
                 ('BAGHDAD', 'IRAQ', 'MOSCOW', 'RUSSIA'),
                 ('BAGHDAD', 'IRAQ', 'OTTAWA', 'CANADA'),
                 ('BAGHDAD', 'IRAQ', 'PARIS', 'FRANCE'),
                 ('BAGHDAD', 'IRAQ', 'TOKYO', 'JAPAN'),
                 ('BEIJING', 'CHINA', 'BERLIN', 'GERMANY'),
                 ('BEIJING', 'CHINA', 'KABUL', 'AFGHANISTAN'),
                 ('BEIJING', 'CHINA', 'LONDON', 'ENGLAND'),
                 ('BEIJING', 'CHINA', 'MOSCOW', 'RUSSIA'),
                 ('BEIJING', 'CHINA', 'OTTAWA', 'CANADA'),
                 ('BEIJING', 'CHINA', 'PARIS', 'FRANCE'),
                 ('BEIJING', 'CHINA', 'TOKYO', 'JAPAN'),
                 ('BEIJING', 'CHINA', 'BAGHDAD', 'IRAQ'),
                 ('

Word Movers distance
^^^^^^^^^^^^^^^^^^^^

You'll need the optional ``pyemd`` library for this section, ``pip install pyemd``.

Let's start with two sentences:



In [ ]:
pip install pyemd

     |████████████████████████████████| 91 kB 2.4 MB/s eta 0:00:01
  Created wheel for pyemd: filename=pyemd-0.5.1-cp37-cp37m-macosx_10_9_x86_64.whl size=80086 sha256=c806a9f1b61d3a18372718b06cf6baa962cf405f9ad5110fc9dbe725f8fa0274
  Stored in directory: /Users/chenyuening/Library/Caches/pip/wheels/91/ed/01/9f0beae80e84fc5053f69df3bf956308f2f1ca14b0bf422c32
Successfully built pyemd
Note: you may need to restart the kernel to use updated packages.


In [ ]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

Remove their stopwords.




In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
sentence_obama = [w for w in sentence_obama if w not in STOPWORDS]
sentence_president = [w for w in sentence_president if w not in STOPWORDS]

Compute the Word Movers Distance between the two sentences.



In [ ]:
distance = wv.wmdistance(sentence_obama, sentence_president)
print(f"Word Movers Distance is {distance} (lower means closer)")

2021-03-03 11:10:39,998 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-03-03 11:10:39,999 : INFO : built Dictionary(8 unique tokens: ['illinois', 'media', 'obama', 'speaks', 'chicago']...) from 2 documents (total 8 corpus positions)


'Word Movers Distance is 1.394372053848207 (lower means closer)'


That's all! You've made it to the end of this tutorial.


